In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import json
import os

from tqdm import tqdm

In [2]:
import nltk
from nltk.corpus import stopwords

import spacy
import gensim

# Load Model

In [3]:
# Load nlp model
nlp = spacy.load('en_core_web_sm')

# Load Dataset

In [4]:
with open("meta-data.json", "r") as file:
    data = json.load(file)

# data

In [5]:
df = pd.DataFrame.from_dict(data, 'index')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviewer_id       100 non-null    int64 
 1   review_time       100 non-null    object
 2   rating            100 non-null    int64 
 3   review            100 non-null    object
 4   review_processed  100 non-null    object
 5   aspect_sentiment  100 non-null    object
dtypes: int64(2), object(4)
memory usage: 5.5+ KB
None


,reviewer_id,review_time,rating,review,review_processed,aspect_sentiment
0,1,2024-09-20,1,Why does it look like someone spit on my food?...,"I had a normal transaction, but now I don't wa...","[{'term': 'food', 'class': 'negative', 'probab..."
1,2,2024-12-15,4,It'd McDonalds. It is what it is as far as the...,It's what it is as far as the food and atmosph...,"[{'term': 'staff', 'class': 'positive', 'proba..."
2,3,2024-12-15,1,Made a mobile order got to the speaker and che...,I made a mobile order got to the speaker and c...,"[{'term': 'speaker', 'class': 'neutral', 'prob..."
3,4,2024-11-20,5,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer s...,"[{'term': 'service', 'class': 'positive', 'pro..."
4,5,2024-10-20,1,"I repeat my order 3 times in the drive thru, a...",I repeat my order three times in the drive thr...,"[{'term': 'fries', 'class': 'positive', 'proba..."


# Modeling

In [6]:
# Load Bing Liu's opinion word dictionary
bing_liu_opinion_words = set()  # Add the actual list of opinion words here

# Function to load opinion words from Bing Liu lexicon
def load_opinion_words(filepath):
    global bing_liu_opinion_words
    temp = pd.read_table(filepath, comment=';', header=None)[0].to_list()
    bing_liu_opinion_words = bing_liu_opinion_words.union(set(temp))


# Load opinion words
current_dir = os.getcwd()
load_opinion_words(os.path.join(current_dir, 'util/opinion-lexicon-English/negative-words.txt'))
load_opinion_words(os.path.join(current_dir, 'util/opinion-lexicon-English/positive-words.txt'))

In [7]:
corpus = df['review_processed'].values

In [8]:
# Define the list of stopwords
stop_words = set(stopwords.words('english'))

In [9]:
# Preprocessing text
def preprocessing(text):

    # Get token of words
    doc = nlp(text)
    result = []
    for token in doc:
        t = token.lemma_.lower()

        if re.match(r'^[0-9\W]+$', t) or len(t) < 3 or t in stop_words:
            continue
        # If the token is adjective, noun, propn, or verb
        if token.pos_ in ['NOUN', 'PROPN', 'VERB']:
            result.append(t)
        # elif token.pos_ in ['ADJ', 'VERB']:
        #     result.append(t)
        # If the token is ADJ but not sentiment opinion
        elif token.pos_ in ['ADJ'] and t not in bing_liu_opinion_words:
            result.append(t)
        else:
            continue
        # result.append(t)
    return result

# Create texts
texts = [preprocessing(document) for document in corpus]

# Create dictionary
dictionary = gensim.corpora.Dictionary(texts)


# Convert documents into Bag-of-words format
corpus_bow = [dictionary.doc2bow(text) for text in texts]

# Train the TF-IDF model
tfidf_model = gensim.models.TfidfModel(corpus_bow)

# Get corpus tfidf 
corpus_tfidf = tfidf_model[corpus_bow]

In [10]:
def topic_model_coherence_generator(corpus, texts, dictionary,
                                    start_topic_count=2, end_topic_count=10,
                                    step=1, cpus=1):
    models = []
    coherence_scores = []
    for topic_nums in tqdm(range(start_topic_count, end_topic_count+1, step)):
        lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary,
                                           chunksize=1740, alpha='auto',
                                           eta='auto', random_state=42,
                                           iterations=500, num_topics=topic_nums,
                                           passes=20, eval_every=None)

        cv_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model,
                                                                     corpus=corpus,
                                                                     texts=texts,
                                                                     dictionary=dictionary,
                                                                     coherence='c_v')
        coherence_score = cv_coherence_model_lda.get_coherence()
        coherence_scores.append(coherence_score)
        models.append(lda_model)


    return models, coherence_scores

models, coherence_scores = topic_model_coherence_generator(corpus=corpus_tfidf,
                                                           texts=texts,
                                                           dictionary=dictionary)
opt_model = models[np.argmax(coherence_scores)]

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:38<00:00,  4.30s/it]


In [11]:
opt_model = models[2]

In [12]:
# Calculate overall mean coherence score
topics_coherences = opt_model.top_topics(corpus_tfidf, topn=20)

In [13]:
coherence_scores

[0.39835983573199785,
 0.48056923951398706,
 0.518990126501683,
 0.4962744441652319,
 0.45629886386610874,
 0.5337222806693261,
 0.512233069565416,
 0.5331722324753213,
 0.48853539318880923]

# Result

In [14]:
# Visualize result: Topic with weights

topics_with_wts = [item[0] for item in topics_coherences]
print("LDA Topics with Weights")
print('='*50)
for idx, topic in enumerate(topics_with_wts):
  print(f'Topic {idx + 1}:')
  print([(term, round(wt, 3)) for wt, term in topic])
  print()

LDA Topics with Weights
Topic 1:
[('get', 0.011), ('order', 0.01), ('drive', 0.009), ('customer', 0.008), ('service', 0.008), ('ask', 0.008), ('line', 0.008), ('thru', 0.007), ('long', 0.007), ('drink', 0.007), ('mcdona', 0.007), ('wait', 0.006), ('time', 0.006), ('busy', 0.006), ('extra', 0.005), ('food', 0.005), ('experience', 0.005), ('wrap', 0.005), ('window', 0.005), ('hang', 0.005)]

Topic 2:
[('mcdonald', 0.014), ('food', 0.008), ('smile', 0.007), ('fry', 0.007), ('people', 0.007), ('order', 0.007), ('answer', 0.007), ('need', 0.006), ('mean', 0.006), ('see', 0.006), ('attitude', 0.006), ('get', 0.006), ('meal', 0.006), ('time', 0.006), ('window', 0.006), ('day', 0.005), ('mess', 0.005), ('plain', 0.005), ('manager', 0.005), ('bag', 0.005)]

Topic 3:
[('service', 0.01), ('get', 0.008), ('take', 0.008), ('ice', 0.007), ('order', 0.007), ('food', 0.007), ('cashier', 0.007), ('wait', 0.006), ('time', 0.006), ('3rd', 0.006), ('card', 0.006), ('eat', 0.006), ('person', 0.006), ('tast

In [15]:
topics = [[(term, round(wt, 3))
              for term, wt in opt_model.show_topic(n, topn=20)]
          for n in range(0, opt_model.num_topics)
          ]

topic_df = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics],
                       columns=['Term per Topic'],
                       index=[str(t) for t in range(1, opt_model.num_topics+1)])
topic_df

,Term per Topic
1,"food, know, job, place, fill, order, staff, sm..."
2,"service, get, take, ice, order, food, cashier,..."
3,"mcdonald, food, smile, fry, people, order, ans..."
4,"get, order, drive, customer, service, ask, lin..."


In [16]:
topic_json = topic_df.reset_index().rename({'index': 'topic', 'Term per Topic': 'term'}, axis=1).to_dict('records')

topic_json

[{'topic': '1',
  'term': 'food, know, job, place, fill, order, staff, small, normal, say, coffee, drink, store, cup, take, chill, saturday, able, professional, caffeine'},
 {'topic': '2',
  'term': 'service, get, take, ice, order, food, cashier, wait, time, 3rd, card, eat, person, taste, new, inside, upgrade, kitchen, make, cookie'},
 {'topic': '3',
  'term': 'mcdonald, food, smile, fry, people, order, answer, need, mean, see, attitude, get, meal, time, window, day, mess, plain, manager, bag'},
 {'topic': '4',
  'term': 'get, order, drive, customer, service, ask, line, thru, long, drink, mcdona, wait, time, busy, extra, food, experience, wrap, window, hang'}]

In [17]:
# Interpreting result
tm_results = opt_model[corpus_tfidf]

# Corpus Topics
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0]
                 for topics in tm_results]

corpus_topics[:5]

[(0, 0.9071873),
 (2, 0.8879401),
 (1, 0.92724097),
 (1, 0.8864034),
 (2, 0.9397753)]

In [18]:
count = 0
for topics in tm_results:
    count += 1
    temp = [t[-1] for t in topics]
    if count == 2:
        break
print(temp)
sum(temp)

[0.028695045, 0.041521836, 0.8879401, 0.041842982]


0.9999999720603228

In [19]:
corpus_topic_df = pd.DataFrame(index=df['reviewer_id'].values)
corpus_topic_df['topic'] = [item[0]+1 for item in corpus_topics]
corpus_topic_df['probability'] = [round(item[1], 4) for item in corpus_topics]
corpus_topic_df['terms'] = [topic_df.iloc[t[0]]['Term per Topic'] for t in corpus_topics]
# corpus_topic_df['reviewer_id'] = df['reviewer_id'].values

corpus_topic_df.head()

,topic,probability,terms
1,1,0.9072,"food, know, job, place, fill, order, staff, sm..."
2,3,0.8879,"mcdonald, food, smile, fry, people, order, ans..."
3,2,0.9272,"service, get, take, ice, order, food, cashier,..."
4,2,0.8864,"service, get, take, ice, order, food, cashier,..."
5,3,0.9398,"mcdonald, food, smile, fry, people, order, ans..."


In [20]:
corpus_topic_json = corpus_topic_df.to_dict('index')

# corpus_topic_json

In [21]:
df['topic'] = df['reviewer_id'].apply(lambda x: corpus_topic_json[x])

df.head()

,reviewer_id,review_time,rating,review,review_processed,aspect_sentiment,topic
0,1,2024-09-20,1,Why does it look like someone spit on my food?...,"I had a normal transaction, but now I don't wa...","[{'term': 'food', 'class': 'negative', 'probab...","{'topic': 1, 'probability': 0.9071999788284302..."
1,2,2024-12-15,4,It'd McDonalds. It is what it is as far as the...,It's what it is as far as the food and atmosph...,"[{'term': 'staff', 'class': 'positive', 'proba...","{'topic': 3, 'probability': 0.8878999948501587..."
2,3,2024-12-15,1,Made a mobile order got to the speaker and che...,I made a mobile order got to the speaker and c...,"[{'term': 'speaker', 'class': 'neutral', 'prob...","{'topic': 2, 'probability': 0.9272000193595886..."
3,4,2024-11-20,5,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer s...,"[{'term': 'service', 'class': 'positive', 'pro...","{'topic': 2, 'probability': 0.8863999843597412..."
4,5,2024-10-20,1,"I repeat my order 3 times in the drive thru, a...",I repeat my order three times in the drive thr...,"[{'term': 'fries', 'class': 'positive', 'proba...","{'topic': 3, 'probability': 0.9398000240325928..."


In [22]:
result = df.to_dict('index')
# result

# Save Result

In [23]:
# with open("meta-topic.json", "w") as file:
#     json.dump(corpus_topic_json, file, indent=4)

In [24]:
# with open("map-topic.json",  "w") as file:
#     json.dump(topic_json, file, indent=4)